Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
Обернуть его в Dataloader
Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import math
import torch

import numpy as np

from PIL import Image
from torchvision import transforms, datasets

import torch.nn.functional as F
import torch.nn as nn

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
data_x, data_y = fetch_california_housing(return_X_y=True, as_frame=True)

In [3]:
data_x_train, data_x_test, data_y_train, data_y_test = train_test_split(data_x, data_y,
                                                                  test_size=0.25,
                                                                  random_state=13)

In [4]:
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(data_x_train.astype(np.float32).to_numpy()),
                                               torch.from_numpy(data_y_train.astype(np.float32).to_numpy()))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256)

test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(data_x_test.astype(np.float32).to_numpy()),
                                               torch.from_numpy(data_y_test.astype(np.float32).to_numpy()))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256)

In [5]:
print(data_x_train)

       MedInc  HouseAge   AveRooms  AveBedrms  Population  AveOccup  Latitude  \
5707   3.5174      36.0   4.547945   1.094368      1357.0  2.065449     34.21   
3754   2.9728      36.0   4.299465   0.997326      1217.0  3.254011     34.18   
11866  1.6944      11.0  21.372093   4.627907        69.0  1.604651     40.19   
19325  3.7143      49.0   6.201087   1.298913       505.0  2.744565     38.53   
1962   2.9219      17.0   6.113960   1.128205       862.0  2.455840     38.73   
...       ...       ...        ...        ...         ...       ...       ...   
153    4.7708      52.0   6.727700   1.075117       612.0  2.873239     37.81   
866    5.2879      12.0   5.410596   1.006623      3436.0  3.250710     37.57   
74     2.4830      20.0   6.278195   1.210526       290.0  2.180451     37.81   
14512  6.0891       5.0   5.469595   0.918919      1063.0  3.591216     32.91   
338    2.2467      46.0   5.940678   1.104520      1339.0  3.782486     37.74   

       Longitude  
5707    

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
        self.drop2 = nn.Dropout(0.3)
        self.batch1 = nn.BatchNorm1d(128)
        self.batch2 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batch1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.batch2(x)
        x = self.drop2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

net = Net()

In [7]:
optimizer_func = [torch.optim.RMSprop(net.parameters(), lr=0.001, eps=1e-07), torch.optim.SGD(net.parameters(), lr=0.01), torch.optim.Adam(net.parameters())]
criterion = nn.MSELoss()

In [8]:
len(optimizer_func)

3

In [11]:
result = dict()
optimizer_list = ['RMSprop', 'SGD', 'Adam']
for i in range(3):
    func = optimizer_list[i]
    optimizer = optimizer_func[i]
    for epoch in tqdm(range(100)):  
        running_loss = 0.0
        epoch_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0], data[1]
            
            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += outputs.shape[0] * loss.item()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if i % 300 == 0:    
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    print(epoch, epoch_loss / len(train_dataset))
    result[func] = epoch + 1, epoch_loss / len(train_dataset)

print('Training is finished!')
# print(epoch, epoch_loss / len(train_dataset))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 0.003


C:\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([120])) that is different to the input size (torch.Size([120, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  2%|█▋                                                                                | 2/100 [00:00<00:19,  5.11it/s]

[2,     1] loss: 0.003
[3,     1] loss: 0.003


  3%|██▍                                                                               | 3/100 [00:00<00:17,  5.54it/s]

...

[98,     1] loss: 0.003
[99,     1] loss: 0.003


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:18<00:00,  5.56it/s]

[100,     1] loss: 0.003


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.40it/s]

99 5.115353797202887
Training is finished!


In [ ]:
print(result)

{'RMSprop': (100, 1.3281817846520003), 'SGD': (100, 1.3255716610324475), 'Adam': (100, 1.3250196221570945)}
